# Обучение моделей в Pytorch

In [ ]:
import os
from os.path import join as pjoin
from shutil import rmtree

import albumentations
import numpy as np
import torch
from PIL import Image
from accelerate import Accelerator
from albumentations.pytorch.transforms import ToTensorV2
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataset import CustomVOCSegmentation
from train import CheckpointSaver, load_checkpoint, train
from unet import UNet, count_model_params

## Агументации

Трансформации/аугментации для исходных изображений и масок/таргетов.

Аугментации для задач компьютерного хрения: https://albumentations.ai/

In [ ]:
IMAGE_SIZE = 512
transforms = albumentations.Compose(
    [
        albumentations.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
        albumentations.AdvancedBlur(p=0.5),
        albumentations.GaussNoise(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.CLAHE(p=0.5),
        albumentations.RandomBrightnessContrast(p=0.5),
        albumentations.RandomGamma(p=0.5),
        albumentations.ColorJitter(p=0.5),
        ToTensorV2(),
    ]
)

## Dataset

Набор данных Pascal VOC. Рассмотрим его версию для задачи сегментации. 

Сайт: http://host.robots.ox.ac.uk/pascal/VOC/

Лидерборд за 2012 год: http://host.robots.ox.ac.uk:8080/leaderboard/displaylb_main.php?challengeid=11&compid=5

In [ ]:
train_dataset = datasets.VOCSegmentation(
    root="data",
    year="2012",
    image_set="train",
    download=True,
    transforms=transforms,
)

val_dataset = datasets.VOCSegmentation(
    root="data",
    year="2012",
    image_set="val",
    download=True,
    transforms=transforms,
)

In [ ]:
len(train_dataset), len(val_dataset)

In [ ]:
image, target = train_dataset[0]
Image.fromarray(image.numpy().transpose(1, 2, 0))

Проблема!

См. `dataset.py`

In [ ]:
train_dataset = CustomVOCSegmentation(
    root="data",
    year="2012",
    image_set="train",
    download=False,
    transform=transforms,  # transform!
)

val_dataset = CustomVOCSegmentation(
    root="data",
    year="2012",
    image_set="val",
    download=False,
    transform=transforms,  # transform!
)

In [ ]:
image, target = train_dataset[0]
Image.fromarray(image.numpy().astype(np.uint8).transpose(1, 2, 0))

In [ ]:
Image.fromarray(255 * target[15, :, :].numpy().astype(np.uint8))

## UNet model

![UNet](unet.jpg)

Визуализация разных типов сверток: https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md

См. `unet.py`

In [ ]:
model = UNet(in_channels=3, out_channels=21)
print(model)

In [ ]:
count_model_params(model)

## Accelerator

"Accelerate — это библиотека, которая позволяет запускать один и тот же код PyTorch в любой распределенной конфигурации, добавляя всего четыре строки кода! Короче говоря, обучение и вывод в больших масштабах стали простыми, эффективными и адаптируемыми". (c)

Сайт: https://huggingface.co/docs/accelerate/index

In [ ]:
accelerator = Accelerator(cpu=False, mixed_precision="fp16")

## Checkpointer

Класс для сохранения наилучших версий модели в процессе обучения.

См. класс `Checkpointer` в `train.py`

## Обучаем модель

См. `train.py`

In [ ]:
LEARNING_RATE = 1e-4
BATCH_SIZE = 4
NUM_WORKERS = 2
EPOCH_NUM = 1
CHECKPOINTS_DIR = "checkpoints"
TENSORBOARD_DIR = "tensorboard"
RM_CHECKPOINTS_DIR = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True
)
val_dataloader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True
)

model = UNet(in_channels=3, out_channels=21)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=5, gamma=0.8)
metric_fn = loss_fn

if RM_CHECKPOINTS_DIR:
    rmtree(CHECKPOINTS_DIR, ignore_errors=True)

os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
checkpointer = CheckpointSaver(
    accelerator=accelerator,
    model=model,
    metric_name="metric",
    save_dir=CHECKPOINTS_DIR,
    max_history=5,
    should_minimize=True,
)

In [ ]:
# !pip install tensorboard
# tensorboard_logger = None

os.makedirs(TENSORBOARD_DIR, exist_ok=True)
tensorboard_logger = torch.utils.tensorboard.SummaryWriter(log_dir=TENSORBOARD_DIR)

In [ ]:
# акселерируем
model, optimizer, train_dataloader, val_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, val_dataloader, lr_scheduler
)

In [ ]:
train(
    model=model,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    loss_function=loss_fn,
    metric_function=metric_fn,
    lr_scheduler=lr_scheduler,
    accelerator=accelerator,
    epoch_num=EPOCH_NUM,
    checkpointer=checkpointer,
    tb_logger=tensorboard_logger,
    save_on_val=True,
)

## Загрузим и протестируем обученную модель

Предобученный чекпоинт: https://disk.yandex.ru/d/C6dRX7Un1L7qsw

Поместить в ".\checkpoints"

In [ ]:
model = UNet(in_channels=3, out_channels=21)
model = load_checkpoint(model=model, load_path=pjoin(CHECKPOINTS_DIR, "model_checkpoint_best.pt"))
model = model.to(DEVICE)

In [ ]:
sample_idx = 42
image, target = val_dataset[sample_idx]
target = torch.argmax(target, axis=0)
preds = torch.argmax(F.softmax(model(image.unsqueeze(0).to(DEVICE)), dim=1).squeeze(0), axis=0)

fig, ax = plt.subplots(1, 3, figsize=(9, 18));
ax[0].imshow(image.numpy().transpose(1, 2, 0).astype(np.uint8));
ax[1].imshow(target.numpy());
ax[2].imshow(preds.cpu().numpy());

## Вопросы/задание на оставшееся время

- Каким образом можно сделать так, чтобы модель предсказывала лишь бинарную маску, на которой отмечены люди?
- Что нам нужно сделать, чтобы дообучить эту модель предсказывать только людей на том же самом датасете?
- Реализуйте самостоятельно